### Medidas de desempeño en el caso de nodos con tasa de servicio dependiente de la carga.  
Recordemos que en una red cerrada de  Gordon-Newell en estado de equilibrio con tasa de servicio dependiente de la carga $\mu_i(j)$ en el nodo $i$ y $\{e_i\}$ conjunto de soluciones de las ecuaciones de balance de flujo, se cumple:

$$\Pi(\vec{n}) = \Pi(n_1,\cdots,n_M) = \frac{1}{G} \prod_{i=1}^M x_i(n_i), \qquad \text{con}\qquad x_i(n_i) =  \frac{e_i^{n_i}}{\prod_{j=1}^{n_i} \mu_i(j)}, \qquad\qquad \sum_{i=1}^M n_i= K$$

y G es una constante de normalización definida por:

$$G = \sum_{\vec{n} \in S} \prod_{i=1}^M x_i(n_i)$$

En lo que sigue derivaremos un método para calcualr la constante $G(K,M)$ para este caso general.

### Algoritmo de convolución

Consideremos la siguiente función generatriz:
$$f(z) = f_1(z) f_2(z) \cdots f_M(z) \qquad \text{ con } \qquad f_i(z) = \sum_{k=0}^{\infty} x_i(k) z^k$$

Esta serie de potencias infinita en $z$ tiene coeficientes en $z^k$ que son la suma de todos los productos de la forma:

$$x_1(n_1) x_2(n_2) \cdots x_M(n_m) \qquad \text{ tales que } \qquad  \sum_{i=1}^{M} n_i = k$$

es decir sobre el espacio de estados $S(k,M)$, de modo que podemos escribir_

$$ f(z) = 1+ G(1,M)z^1 + G(2,M)z^2 + \cdots + G(k,M)z^k + \cdots $$

Así, la función generatriz puede construirse iterativamente como sigue:
\begin{eqnarray}
g_1(z) & =&f_1(z)\\
&&\\
g_i(z) & = & g_{i-1}(z) f_i(z), \qquad i=2, 3, \cdots, M\\
\end{eqnarray}

donde el coeficiente que acompaña a $z^k$ en $g_i(z)$ es exactamente $G(k,i)$

Igualando coeficientes de $z^k$ en estas ecuaciones, se tiene:
\begin{eqnarray}
G(k,1) & =&x_1(k)\\
&&\\
G(k,i) & = & \sum_{j=0}^k G(j,i-1)x_i(k-j)  \qquad i=2, 3, \cdots, M\\
\end{eqnarray}

con condición de borde:
$$G(0,i)=1 \qquad\forall i=1,\cdots, M$$

Con lo que tenemos un algoritmo recursivo para obtener $G(K,M)$ que es mucho mas rápido que sumar los productos $\prod_{i=1}^M x_i(n_i)$ sobre todo el espacio de estados $S(K,M)$.

**Ejercicio:** Considere un sistema cliente-servidor con  $m$ clientes y un servidor con una CPU y dos discos de almacenamiento. El sistema 
se puede modelar como la red cerrada de colas de la figura,
en que el nodo de clientes se caracteriza por una tasa de servicio dependiente de la carga, dando cuenta del número de clientes en estado de reflexión.

<img src="modelo5.png" width="500">

Considere el caso simplificado en que $m=10$, las probabilidades de enrutamiento $p=0.5$ y $q=0.4$,  el tiempo  de reflexión de las estaciones cliente se distribuye exponencial de tasa $\mu_C=0.1[req/seg]$, 
que los tiempos de servicio de la CPU y los discos también son exponenciales de tasas $\mu_{CPU}= 4[req/seg]$,
$\mu_{A}=0.2[req/seg]$ y $\mu_{B}=0.4[req/seg]$. 

**(a)** Calcule el número de estados posibles  de la red y dibuje el grafo de la red 

**(b)** Escriba el sistema de ecuaciones de balance de flujo y obtenga valores para $\{e_i\}_{i=1,2,3,4}$

**(c)** Calcule la constante para el cálculo de las probabilidades en el equilibrio, considerando el algoritmo de convolución

**(d)** Calcule las probabilidades en el equilibrio, el número medio de trabajos y utilización de  cada nodo 

**(e)** Para el cálculo de los tiempos de respuesta en cada nodo, considere que para los nodos de tasa de servicio fija, se cumple:
$$\lambda_i = \mu_i U_i$$
Utilice lo anterior para calcular las tasas de proceso y tiempos de respuesta de cada nodo. 

**Resolución:** 

**(a)** El grafo asociado es:
<img src="modelo5a.png" width="400">

**(b)** las ecuaciones de balance de flujo quedan:
$$\begin{eqnarray}
\lambda_{CPU} & = & \lambda_A + \lambda_B + \lambda_C\\
&&\\
\lambda_A & = & p (1-q) \lambda_{CPU} \\
&&\\
\lambda_B & = & p q \lambda_{CPU} \\
&&\\
\lambda_C & = & (1-p)\lambda_{CPU} \\
\end{eqnarray}
$$


Una solución posible de este sistema es $e_{CPU}= 1, e_A = 0.3, e_B = 0.2, e_C = 0.5$.

El número de estados posibles en la red es:
$$\frac{(K+M-1)!}{K! (M-1)!} = \frac{13*12*11}{2*3} = 143*2 = 286$$

In [4]:
#(c) calculo de constante G, dadas las tasas de servicio y de proceso de cada nodo
K=10
e<- c(0.5,1,0.3,0.2)
u <- c(0.1,4,0.2,0.4)

#cálculo de x_i(n_i), i=1...4, n_i=0,1,...K
x <- matrix(0,nrow=K+1,ncol=4)
x[1,1:4]<-rep(1,4)
x[2,1:4] <- e/u
for (k in 2:K){
    x[k+1,1:4]<- x[k,1:4]*(e/c(u[1]*k,u[2],u[3],u[4]))
}
print(x)

#Cálculo de G mediante algoritmo de convolución
G <- matrix(0,nrow=K+1,ncol=4)
#condiciones iniciales
G[1,1:4]<-x[1,1:4]
G[1:K+1,1] <- x[1:K+1,1]
#convolución
for (i  in 2:4){
    for (k in 1:K){
        sum <- 0
        for (j in 0:k){
          sum <- sum + G[j+1,i-1]*x[k+1-j,i]
        }
        G[k+1,i]<- sum
    }
}
print(G)
#constante de normalización
GG <- G[K+1,4]
print(GG)

           [,1]         [,2]     [,3]         [,4]
 [1,]  1.000000 1.000000e+00  1.00000 1.0000000000
 [2,]  5.000000 2.500000e-01  1.50000 0.5000000000
 [3,] 12.500000 6.250000e-02  2.25000 0.2500000000
 [4,] 20.833333 1.562500e-02  3.37500 0.1250000000
 [5,] 26.041667 3.906250e-03  5.06250 0.0625000000
 [6,] 26.041667 9.765625e-04  7.59375 0.0312500000
 [7,] 21.701389 2.441406e-04 11.39062 0.0156250000
 [8,] 15.500992 6.103516e-05 17.08594 0.0078125000
 [9,]  9.688120 1.525879e-05 25.62891 0.0039062500
[10,]  5.382289 3.814697e-06 38.44336 0.0019531250
[11,]  2.691144 9.536743e-07 57.66504 0.0009765625
           [,1]      [,2]       [,3]       [,4]
 [1,]  1.000000  1.000000    1.00000    1.00000
 [2,]  5.000000  5.250000    6.75000    7.25000
 [3,] 12.500000 13.812500   23.93750   27.56250
 [4,] 20.833333 24.286458   60.19271   73.97396
 [5,] 26.041667 32.113281  122.40234  159.38932
 [6,] 26.041667 34.069987  217.67350  297.36816
 [7,] 21.701389 30.218886  356.72914  505.41322
 [8,

In [5]:
# (d) Cálculo medidas de desempeño
#mat matriz de todos los estados S(K,M), uno por fila
#proba, probabilidad en el equilibrio para cada estado
mat <- c(K,0,0,0)
proba <- x[K+1,1]/GG 

#L y U vectores para calculo iterativo de L y U para los 4 nodos
L <- c(K*proba,0,0,0)
U <- c(proba,0,0,0)

#Generación de todos los estados en S(K,M)= {(i,j,k,l) enteros que cumplen i+j+k+l=K}
for (i in 0:(K-1)){
    limj= max(0, K-i)
    for (j in 0:limj){
        limk = max(0,K-i-j)
        for (k in 0:limk){
            l = K-i-j-k
            if (l>=0){
                mat <- rbind(mat,c(i,j,k,l))
                p<- (x[i+1,1] * x[j+1,2] * x[k+1,3] * x[l+1,4])/GG
                proba <- c(proba, p)
                L <- L + rep(p,4)*c(i,j,k,l)
                if (i>0)
                    U[1] <- U[1] + p
                if (j>0)
                    U[2]<- U[2] + p
                if (k>0)
                    U[3] <- U[3] + p
                if (l>0)
                    U[4] <- U[4] + p
            }
        }
    }
}
print(sum(proba))
print(cbind(mat, proba))
print(rbind(L,U))

#(e) Cáculo de tasas de proceso y tiempos de respuesta de los 4 nodos 
factorT <- u[2]*U[2]
tasa <- rep(factorT,4)*e
R <- L/tasa
print(rbind(tasa,R))

        

[1] 1
                       proba
mat 10  0  0  0 9.292473e-04
     0  0  0 10 3.372053e-07
     0  0  1  9 1.011616e-06
     0  0  2  8 3.034847e-06
     0  0  3  7 9.104542e-06
     0  0  4  6 2.731363e-05
     0  0  5  5 8.194088e-05
     0  0  6  4 2.458226e-04
     0  0  7  3 7.374679e-04
     0  0  8  2 2.212404e-03
     0  0  9  1 6.637211e-03
     0  0 10  0 1.991163e-02
     0  1  0  9 1.686026e-07
     0  1  1  8 5.058079e-07
     0  1  2  7 1.517424e-06
     0  1  3  6 4.552271e-06
     0  1  4  5 1.365681e-05
     0  1  5  4 4.097044e-05
     0  1  6  3 1.229113e-04
     0  1  7  2 3.687339e-04
     0  1  8  1 1.106202e-03
     0  1  9  0 3.318606e-03
     0  2  0  8 8.430131e-08
     0  2  1  7 2.529039e-07
     0  2  2  6 7.587118e-07
     0  2  3  5 2.276135e-06
     0  2  4  4 6.828406e-06
     0  2  5  3 2.048522e-05
     0  2  6  2 6.145566e-05
     0  2  7  1 1.843670e-04
     0  2  8  0 5.531009e-04
     0  3  0  7 4.215066e-08
     0  3  1  6 1.264520e-07
     0  

       [,1]      [,2]      [,3]      [,4]
L 3.3102212 0.1978343 6.0021006 0.4898439
U 0.9641594 0.1655111 0.9930664 0.3310221
           [,1]      [,2]       [,3]      [,4]
tasa  0.3310221 0.6620442  0.1986133 0.1324088
R    10.0000000 0.2988234 30.2200381 3.6994798


## Análisis del Valor Medio
El algoritmo del valor medio se puede extender al caso de nodos con tasa de servicio dependiente de la carga, de la manera siguiente.

- La ecuación para la tasa de proceso sigue siendo válida:

$$T(K)  =  \frac{K}{\sum_{i=1}^M v_i R_i }$$

- Sean $\pi_i(j \mid k)$ la proporción de tiempo que el nodo $i$ tiene $j$ trabajos cuando la red completa tiene $k$ trabajos (en el equilibrio).

Intuitivamente se puede esperar que un trabajo que llega al nodo $i$ y encuentra $j-1$ trabajos en ese nodo,  tendrá un tiempo medio de respuesta igual a:

$$\frac{j}{\mu_i(j)}$$

Luego podemos escribir:

$$ R_i(k) = \sum_{j=1}^k \pi_i(j-1 \mid k-1) \frac{j}{\mu_i(j)}$$

Y el número medio de trabajos será:

$$L_i(k) = \sum_{j=1}^k j\pi_i(j \mid k)$$

Por definición:
$$\pi_i(0\mid 0) =1$$

Y para $k>0$ es posible determinar los $\pi_i(j\mid k)$ como sigue:

$$ \pi_i(j\mid k) = \left\{ \begin{array}{ll}
\frac{v_iT(k)}{\mu_i(j)} \pi_i(j-1 \mid k-1) & j=1,\cdots,k\\
&\\
1 - \sum_{l=1}^k \pi_i(l \mid k) & j=0
\end{array} \right. $$

Así tenemos un sistema de ecuaciones que nos permite calcular iterativamente en $k=0,\cdots,K$:

$$\pi_i(j \mid k)_{j=0,\cdots,k},\qquad R_i(k),\qquad T(k),\qquad L_i(k), \qquad i=1,\cdots,M$$


Notar además que en este caso, las utilizaciones en cada nodo se pueden calcular como
$$U_i(k) = \sum_{j=1}^k \pi_i(j \mid k)\qquad i=1,\cdots,M$$


y las tasas de proceso  por:
$$\lambda_i(k) = T(k) v_i\qquad i=1,\cdots,M$$


**Propuesto:** Utilice estas ecuaciones de recurrencia para resolver las preguntas (d) y (e) del ejercicio anterior.

**Resolución:** Transformando la red cerrada en una abierta, nos queda:
<img src="modelo5b.png" width="500">

de donde:

$$v_0 = 1  = v_C  \implies v_C = 1$$ 

Resolviendo para los otros tres nodos (en la red cerrada):

$$\begin{eqnarray}
v_{CPU} & = & \frac{v_C}{1-p} \implies v_{CPU} = 2\\
&&\\
v_B & = &(1-p)q \,v_{CPU} \implies v_{B} = 0.6\\
&&\\
v_C & = & pq\, v_{CPU} \implies v_{C} = 0.4\\
\end{eqnarray}
$$



In [6]:
# ecuacions de recurrencia
K=10
M=4
L <- rep(0,M)
R <- rep(0,M)
U <- rep(0,M)
u <- c(0.1,4,0.2,0.4)
v <- c(1,2,0.6,0.4)
prob <- matrix(1,nrow=K+1,ncol=M)
nprob <- matrix(1,nrow=K+1,ncol=M)
# Cálculo de mu_i(j), i=1,2,3,4 y j=1,...K
mu <- matrix(0,nrow=K,ncol=M)
mu[1,] <- u
for (k in 2:K)
    mu[k,]<- c(u[1]*k,u[2],u[3],u[4])

#algoritmo iterativo para calcular R, T, L y U
for (k in 1:K){   
    R <- rep(0,M)  
    for (j in 1:k){
        R <- R + (prob[j,]*rep(j,M)/mu[j,])
    }
    T <- k/(t(v)%*%R)
    L <- rep(0,M)
    sum <- rep(0,M)
    for (j in 1:k){
        nprob[j+1,] <- rep(T,M)*(v/mu[j,])*prob[j,]
        sum <- sum + nprob[j+1,]
        L <- L + rep(j,M)*nprob[j+1,]
    }
    U <- sum
    nprob[1,] <- rep(1,M) - U
    prob <- nprob  
}
tasa = rep(T,4)*v
print(rbind(L,U,tasa,R))

           [,1]      [,2]       [,3]      [,4]
L     3.3102212 0.1978343  6.0021006 0.4898439
U     0.9641594 0.1655111  0.9930664 0.3310221
tasa  0.3310221 0.6620442  0.1986133 0.1324088
R    10.0000000 0.2988234 30.2200381 3.6994798
